Orquestador de ventas 360: resumen
- Descargar xlsx de sistemas SAI, CAMUNDA, SAGI, ZOHO y PISP. 
- Descargar pdfs de sistema CAMUNDA
Todo lo anterior es la información mínima indispensable para poder generar información de ventas, que será la referencia interna sobre la que desplegaremos el alimentado de reportes y la generación de reglas para dar retro al usuario.

In [ ]:
# Importa librerías y chromedriver
import sys
import os
import glob
import yaml
import pandas as pd
from datetime import datetime
import shutil
import time

# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
Libreria_SAI = os.path.join(script_folder, "Libreria_SAI")
if Libreria_SAI not in sys.path:
    sys.path.append(Libreria_SAI)
Libreria_comunes = os.path.join(script_folder, "Libreria_comunes")
if Libreria_comunes not in sys.path:
    sys.path.append(Libreria_comunes)
Libreria_camunda = os.path.join(script_folder, "Libreria_camunda")
if Libreria_camunda not in sys.path:
    sys.path.append(Libreria_camunda)
Libreria_SAGI = os.path.join(script_folder, "Libreria_SAGI")
if Libreria_SAGI not in sys.path:
    sys.path.append(Libreria_SAGI)

Libreria_PREI = os.path.join(script_folder, "Libreria_PREI")
if Libreria_PREI not in sys.path:
    sys.path.append(Libreria_PREI)
from chrome_driver_load import load_chrome


In [ ]:
# Generar o cargar el archivo con contraseñas
output_yaml = os.path.join(folder_root, "Implementación", "passwords.yaml")
yaml_exists = os.path.exists(output_yaml)

if yaml_exists:
    # Abrir y cargar el contenido YAML en un diccionario
    with open(output_yaml, 'r', encoding='utf-8') as f:
        data_access = yaml.safe_load(f)
    print(f"Archivo cargado correctamente: {os.path.basename(output_yaml)}")
    #print("Contenido:", data_access)
    #sys.exit()

else: 
    platforms = ["SAI", "PREI", "CAMUNDA", "SAGI"]
    fields    = ["password", "user", "link"]
    
    lines = []
    for platform in platforms:
        for field in fields:
            # clave = valor vacío
            lines.append(f"{platform}_{field}: ")
        lines.append("")  # línea en blanco entre bloques
    
    # Escribe el archivo YAML (aunque use "=" tal como en tu ejemplo)
    with open(output_yaml, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    
    print(f"Archivo generado correctamente en: {output_yaml}")
    print(passwords)


# IMSS

## SAI
Esta consola de jupyter lab es se va a encargar de: 
- Descargar órdenes y altas del sistema SAI

In [ ]:
## Descargar SAI
from SAI_download import SAI_download

# Define the available date ranges
today_date = datetime.now().strftime('%d/%m/%Y')
date_ranges = {
    2023: ('01/01/2023', '31/12/2023'), 
    2024: ('01/01/2024', '31/12/2024'), 
    2025: ('01/01/2025', today_date)  # Uses today's date for 2025 end date
}

# Ask the user for a valid year
while True:
    try:
        year_selected = int(input(f"¿Qué año vamos a descargar? {list(date_ranges.keys())}: "))
        if year_selected in date_ranges:
            print(f"\t{year_selected}")
            break
        else:
            print("Invalid year. Please select from the available options.")
    except ValueError:
        print("Invalid input. Please enter a valid year.")

# Retrieve the selected date range
range_date = date_ranges[year_selected]

# Define your download directory
download_directory = os.path.join(folder_root, 'Implementación', 'SAI', f"{year_selected} Temporal")

# Load Chrome with the download directory
driver = load_chrome(download_directory)
SAI_password = data_access.get("SAI_password")
SAI_user = data_access.get("SAI_user")
if driver:
    # Execute the SAI download process with the selected date range
    SAI_download(driver, SAI_user, SAI_password, range_date)


In [ ]:
# Mover los archivos descargados
# Define headers
from openpyxl import load_workbook  # To read metadata from Excel files

Altas_headers = [
    "noAlta", "fechaAltaTrunc", "noContrato", "noOrden", "clave", "cantRecibida", "importe",
    "fpp", "clasPtalRecep", "descUnidad", "clasPtalDist", "descDist", "totalItems", "resguardo"
]

Ordenes_headers = [
    "contrato", "solicitud", "orden", "generacion", "cveArticulo", "fechaExpedicion",
    "lugarEntrega", "descripciónEntrega", "direccionEntrega", "destinoFinal", "estatus",
    "fechaEntrega", "cantidadSolicitada", "precio", "importeSinIva"
]


# Iterate through each year

def get_excel_creation_date(file_path):
    """
    Extracts the real creation date from an Excel file's metadata.
    If metadata is unavailable, falls back to file system modification time.
    """
    try:
        workbook = load_workbook(file_path, read_only=True)
        props = workbook.properties
        created_date = props.created
        workbook.close()  # ✅ esto libera el archivo correctamente
        if created_date:
            return created_date
    except Exception as e:
        print(f"Error reading metadata from {file_path}: {e}")

    return datetime.fromtimestamp(os.path.getmtime(file_path))

for year_processed in date_ranges.keys():
    # Define directories
    Temporal = os.path.join(folder_root, 'Implementación', 'SAI', f"{year_processed} Temporal")
    Final = os.path.join(folder_root, 'Implementación', 'SAI', f"{year_processed} Final")

    # Ensure directories exist
    os.makedirs(Final, exist_ok=True)

    # Get all Excel files in Temporal
    files = [f for f in os.listdir(Temporal) if f.endswith('.xlsx')]

    for file in files:
        file_path = os.path.join(Temporal, file)

        # Read the file headers
        try:
            #df = pd.read_excel(file_path, nrows=1)  # Read only the first row for headers
            with open(file_path, 'rb') as f:
                df = pd.read_excel(f, nrows=1)
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue

        file_headers = list(df.columns)  # Get the headers from the file

        # Determine file type
        if file_headers == Ordenes_headers:
            prefix = "Ordenes"
        elif file_headers == Altas_headers:
            prefix = "Altas"
        else:
            print(f"{file} does not fit either as Altas or Ordenes. Skipping.")
            continue

        # Get file creation date
        # Get file creation date from metadata or system
        file_creation_time = get_excel_creation_date(file_path)
        formatted_date = f"{file_creation_time.year} {file_creation_time.month:02d} {file_creation_time.day:02d}"

        # Define new file name
        new_filename = f"{formatted_date} {prefix}.xlsx"
        new_file_path = os.path.join(Final, new_filename)

        # Move and rename file
        shutil.move(file_path, new_file_path)
        print(f"\t{os.path.basename(Final)}")
        print(f"Moved: {file} -> {new_filename}")

In [ ]:
# Fusionar los archivos descargados
# Ejemplo de uso:
from SAI_processing import merge_SAI_files

SAI_folder = os.path.join(folder_root, 'Implementación', 'SAI')
alta_pivots  = ['noAlta', 'fechaAltaTrunc']
orden_pivots = ['orden', 'fechaExpedicion']
date_regex= r'(\d{4} \d{2} \d{2})'
date_parse_format='%Y %m %d'
Output_filename = "Ordenes_altas.xlsx"
merge_SAI_files(SAI_folder, alta_pivots, orden_pivots, date_regex, date_parse_format, Output_filename)



## PREI

In [ ]:
# Carga librerías y define el año a descargar.
from PREI_downloader import PREI_downloader

PREI_password = data_access.get("PREI_password")
PREI_user = data_access.get("PREI_user") 
# Allowed years
valid_years = {"2023", "2024", "2025"}

In [ ]:
# Loop until a valid year is entered

while True:
    year = input("Enter a year (2023, 2024, 2025): ").strip()
    if year in valid_years:
        break
    else:
        print("Invalid year. Please try again.")

# Create paths using f-strings to substitute the year
dates = os.path.join(folder_root, 'Implementación', 'PREI', f"{year}_dates.xlsx")
temp_folder = os.path.join(folder_root, 'Implementación', 'PREI', f"{year} Temporal")
final_folder = os.path.join(folder_root, 'Implementación', 'PREI', f"{year} Final")
# Check if the Excel file exists
def check_if_exists(element):
    if os.path.exists(element):
        print(f"The file {os.path.basename(element)} exists.")
    else:
        print(f"The file {os.path.basename(element)} does not exist.")
check_if_exists(dates)
check_if_exists(temp_folder)
check_if_exists(final_folder)


In [ ]:
# Descargar, mover y fusionar XLS del PREI

# Ejecutar el ciclo hasta que todos los archivos estén presentes y válidos
from selenium import webdriver
from PREI_merger_and_audit import move_files, merge_files

completed = False
while not completed:
    driver = load_chrome(temp_folder)
    if driver:
        try:
            completed = PREI_downloader(driver, PREI_user, PREI_password, temp_folder, dates)
        finally:
            #input("🧍 Verifica manualmente las descargas y presiona Enter para continuar...")
            driver.quit()
            time.sleep(5) 
print("\nMoviendo archivos")
move_files(temp_folder, final_folder)
time.sleep(5) 
print("\nFusionando archivos")
merge_files(temp_folder, final_folder)
time.sleep(5) 


In [ ]:
# Audita los archivos
from PREI_merger_and_audit import audit
audit(final_folder, year)

### PREI: Fusiona todos los ciclos fiscales

In [ ]:
# Fusiona ciclos fiscales 2023, 2024, 2025
from PREI_merger_and_audit import fusion_2023_2025
# Define expected headers (adjust as needed)
fusion_2023_2025(valid_years, folder_root)

# Camunda - Órdenes

In [ ]:
from CAMUNDA_login import SAI_camunda

download_directory = os.path.join(folder_root,'Implementación', 'CAMUNDA', 'Descargas')

# Initialize the Chrome driver with your custom settings
driver = load_chrome(download_directory)

if driver:
    # Execute the Camunda login and initial automation00
    CAMUNDA_password = data_access.get("CAMUNDA_password")
    CAMUNDA_user = data_access.get("CAMUNDA_user")
    SAI_camunda(driver, CAMUNDA_user, CAMUNDA_password)
    driver.quit()

In [ ]:
from CAMUNDA_merging import CAMUNDA_merging
download_directory = os.path.join(folder_root,'Implementación', 'CAMUNDA', 'Descargas')
# Create the full path to your YAML file.
yaml_file = os.path.join(folder_root, "df_headers.yaml")
# Open and load the YAML file.
with open(yaml_file, "r", encoding="utf-8") as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
INSABI_headers = data.get("columns_INSABI")
duplicados = "NÚMERO DE ORDEN DE SUMINISTRO"
archivo_final = "Camunda 2023-2025.xlsx"
CAMUNDA_merging(download_directory, INSABI_headers, duplicados, archivo_final)


### SAGi


In [14]:
from SAGI_download import SAGI_download
# Initialize Chrome with your custom settings
download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
driver = load_chrome(download_directory)
if driver:
    # Call the function with your login credentials (username and password)
    SAGI_password = data_access.get("SAGI_password")
    SAGI_user = data_access.get("SAGI_user")    
    SAGI_download(driver, SAGI_user, SAGI_password, download_directory)
    driver.quit()


Descargamos facturas 
1)2023-2024 o 
2)2024?
 2



*******************************
Navega a la sección elegida: 2024
El script va a seleccionar 50 resultados por página y continuar...



Please navigate to the 2024 section and press Enter to continue. 


Data extracted and added to output.


Next page button not found or not clickable. Is there only one page of results? (yes/no):  yes


Data for 2024 saved to C:\Users\arman\Dropbox\3. Armando Cuaxospa\Adjudicaciones\Licitaciones 2025\E115 360\Implementación\Descarga de bases\05 23 2024.xlsx
Extraction process completed.


In [16]:
from SAGI_Join_2023_2024 import join_SAGI_files, worksheet_to_df
now = datetime.now()
prefix = f"{now.month:02d} {now.day:02d}"
json_key = os.path.join(folder_root, 'Implementación', 'Key.json')
file2023_2024 = os.path.join(folder_root, 'Implementación', 'SAGI','RAW', f"{prefix} 2023-2024.xlsx")
file_2024 = os.path.join(folder_root, 'Implementación', 'SAGI','RAW', f"{prefix} 2024.xlsx")
output_joined_file = os.path.join(folder_root, 'Implementación', 'SAGI', f"{prefix} 2024 ESTATUS_SAGI.xlsx")

join_SAGI_files(json_key, file2023_2024, file_2024, output_joined_file)

print(f"{'*' * 10} \n archivo generado")

INSABI cargado desde el google sheet

 DATAFRAME PREVIO
<class 'pandas.core.frame.DataFrame'>
Index: 1125 entries, 0 to 2489
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            0 non-null      float64
 1   Número de oficio      0 non-null      float64
 2   Proveedor             1125 non-null   object 
 3   RFC                   1125 non-null   object 
 4   Número de contrato    1125 non-null   object 
 5   Orden de suministro   981 non-null    object 
 6   Número de factura     1125 non-null   int64  
 7   Folio fiscal          1125 non-null   object 
 8   Total                 1125 non-null   object 
 9   CLUES                 981 non-null    object 
 10  Estado de la factura  1125 non-null   object 
 11  Opciones              1125 non-null   object 
 12  Factura               1125 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 123.0+ KB
None

 *****

# Generar consolas